# Site data summary

In [ ]:
# This notebook supports input parameters for automatic report generation. The parameters must be variables in this
# cell, which has a special 'parameters' tag.
DATA_ROOT = r'G:\Shared drives\Covid-19 Spectrum Monitoring\Data'

EXPORT_DATA_ROOT = '' # r'G:\Shared drives\Covid-19 Spectrum Monitoring Data Export'

# The values that override these defaults are in ../config.py
HISTOGRAM_RESOLUTION_SWEEPS = 100
HISTOGRAM_POWER_LOW = -99
HISTOGRAM_POWER_HIGH = -12

TOD_BIN_SIZE = '20T'
DATE_BIN_SIZE = '1D'

median_period='0.25H'

survival_noise_margin_dB = 6

# report.py will make separate reports for each combination of the below parameters
# that exists in the data4t.
site = '09' # 'hospital'

figure_format = 'svg'

In [ ]:
# hide the pink warnings in reports (comment to include them)
import warnings
warnings.simplefilter("ignore")
from environment import *
import figures
import histogram_analysis

def make_bin_size_label(iso_str):
    if iso_str == '1D':
        return '1 day'
    else:
        return iso_str.replace('T', ' min.').replace('D', ' days')

import importlib
figures = importlib.reload(figures)

set_matplotlib_formats(figure_format)

HISTOGRAM_BOUNDS = (HISTOGRAM_POWER_LOW, HISTOGRAM_POWER_HIGH)

SF_BOUNDS = {
    'uplink channels in LTE bands': (-84, HISTOGRAM_POWER_HIGH),
    'downlink channels in LTE bands': (-84, HISTOGRAM_POWER_HIGH),
    'ISM bands': (-77, HISTOGRAM_POWER_HIGH),
    'U-NII bands': (-67, HISTOGRAM_POWER_HIGH)
}

TOD_BIN_SIZE_LABEL = make_bin_size_label(TOD_BIN_SIZE)
DATE_BIN_SIZE_LABEL = make_bin_size_label(DATE_BIN_SIZE)

display(HTML(f'This report was produced {time.strftime(time_format)}'))

In [ ]:
# Source data
hists = histogram_analysis.read_histogram(Path(DATA_ROOT)/'histogram.parquet', site=site)

if EXPORT_DATA_ROOT:
    # this is expensive, so add a flag
    (Path(EXPORT_DATA_ROOT)/'histograms').mkdir(exist_ok=True, parents=True)
    hists.to_csv(Path(EXPORT_DATA_ROOT)/'histograms'/f'{site}.csv.gz')    

# there is slight error in the achieved frequency; map intended nearby frequency
# to achieved frequency
fc_map = dict(zip(np.round(hists.index.levels[0],1),hists.index.levels[0]))

# for 'reasonable' time axis resolution in a heat map
daily_hists = hists.groupby(['Frequency', pd.Grouper(level='Time', freq=DATE_BIN_SIZE)]).sum()

dwell_mode = histogram_analysis.compute_dwell_mode(
    hists,
    rolling_stat_period=median_period,
    by='Frequency'
)

shielded_noise_survival = histogram_analysis.read_histogram(
    Path(DATA_ROOT)/'shielded_noise_survival.parquet'
)

# TODO: this seemed reasonable since the background noise in LTE UL had low variance
# 
# toss histogram counts within 3 dB of the 1-hour median
# lte_ul_threshold = dwell_stats[f'{median_period} Median'].values + 5
# hists_ul_masked = (
#     (hists/hists.sum(axis=1).values[:,None]).mask(hists.columns.values[None,:] <= lte_ul_threshold[:,None])
#     .loc(axis=0)[[fc_map[fc] for fc in fc_lte_ul]]
# )
# time_index_ul = hists_ul_masked.index.get_level_values('Time').floor('10T')
# time_index_ul = time_index-pd.DatetimeIndex(time_index.date)

# hists_by_tod_ul = hists_ul_masked.groupby(['Frequency', time_index]).sum()
# hists_by_tod_ul.index.names = ['Frequency', 'Time']

time_index = hists.index.get_level_values('Time').floor(TOD_BIN_SIZE)
time_index = time_index-pd.DatetimeIndex(time_index.date)

hists_by_tod = hists.groupby(['Frequency', time_index]).sum()
hists_by_tod.index.names = ['Frequency', 'Time']

hists_normed = histogram_analysis.normalize_power_per_dwell(hists, dwell_mode)
hists_normed_total = hists_normed.groupby('Frequency').sum()

hists_total = hists.groupby('Frequency').sum()

summary_table = histogram_analysis.hists_summary_table(hists, site=site)
data_span = f"{summary_table.loc['Start'][0].split(' ')[0]} to {summary_table.loc['End'][0].split(' ')[0]}"
display(summary_table)

# Average power over time

### LTE Uplink Bands

In [ ]:
fc_list = fc_lte_ul
group_name = 'uplink channels in LTE bands'
h_group = histogram_analysis.loc_by_fc_spec(hists, fc_list)

figures = reload(figures)

fc_maxima = np.abs(h_group-1).idxmin(axis=1).groupby('Frequency').max()
fc_min, fc_max = fc_maxima.idxmin().round(1), fc_maxima.idxmax().round(1)

for fc in (fc_min, fc_max):
    fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth, sharey=True)

    group_h_tod = hists_by_tod.loc[fc_map[fc]].sort_index()
    group_h_daily = daily_hists.loc[fc_map[fc]]

    vmax = max(group_h_tod.max().max(), group_h_daily.max().max())

    figures.plot_power_histogram_heatmap(
        group_h_tod,
        title=None,
        bounds=HISTOGRAM_BOUNDS,
        ax=ax1,
        vmax=vmax,
        cbar=False
    )
    ax2.yaxis.get_label().set_visible(False)

    figures.plot_power_histogram_heatmap(
        group_h_daily,
        title=None,#f'Histogram over time, {fc} MHz, site {site}',
        bounds=HISTOGRAM_BOUNDS,
        ax=ax2,
        vmax=vmax
    )
    ax2.yaxis.get_label().set_visible(False)

    # set the metadata, but hide because it's redundant with the caption text
    fig.suptitle(f'Histograms {fc} MHz')
    fig._suptitle.set_visible(False)

    if fc== fc_max:
        reason = f'The peak band power at this frequency was the greatest among {group_name} at this site'
    else:
        reason = f'The peak band power at this frequency was the smallest among {group_name} at this site'

    set_caption(fig,
        f"""
        Histogram heat maps of band power at {fc} MHz from site {site} across the full data time span,
        by time of day ({TOD_BIN_SIZE_LABEL} bin, left) and date ({DATE_BIN_SIZE_LABEL} bin, right). {reason}.
        """
    )

# figures.plot_counts_each_frequency(hists_normed, fc_list, (-20,20));
# xlabel(f'Power relative to 15-min histogram peak (dB')
# title(f'{group_name} relative distribution', visible=False)
# set_caption(
#     gcf(),
#     f"""Band power samples in {group_name} relative to the site noise and interference level at site {site},
#     as gauged by the peak histogram level in each dwell window.
#     """
# )

h_rcum = histogram_analysis.rcumsum(hists.loc[[fc_map[fc] for fc in fc_list]]).sort_index()

fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth)

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=True,
    ax=ax1
);

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=False,
    ax=ax2,
    title=f'Site {site}'
);

ax2.yaxis.get_label().set_visible(False)
ax1.get_legend().set_visible(False)

fig.suptitle(f'Empirical SF {group_name}', visible=False)

set_caption(
    fig,
    f"""Empirical survival functions of band power in {group_name} at site {site}, shown first on log scale
    to illustrate rare events (left)
    and then on a linear scale (right) to emphasize the most frequent power levels. Dotted lines
    indicate expected distortion of the power level by 1\,dB due to noise or compression inside the sensor.
    """
)

### LTE Downlink Bands

In [ ]:
figures = reload(figures)
fc_list = fc_lte_dl
group_name = 'downlink channels in LTE bands'

h_group = histogram_analysis.loc_by_fc_spec(hists, fc_list)

fc_maxima = np.abs(h_group-1).idxmin(axis=1).groupby('Frequency').max()
fc_min, fc_max = fc_maxima.idxmin().round(1), fc_maxima.idxmax().round(1)

for fc in (fc_min, fc_max):
    fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth, sharey=True)

    group_h_tod = hists_by_tod.loc[fc_map[fc]].sort_index()
    group_h_daily = daily_hists.loc[fc_map[fc]]

    vmax = max(group_h_tod.max().max(), group_h_daily.max().max())

    figures.plot_power_histogram_heatmap(
        group_h_tod,
        title=None,
        bounds=HISTOGRAM_BOUNDS,
        ax=ax1,
        vmax=vmax,
        cbar=False
    )
    ax2.yaxis.get_label().set_visible(False)

    figures.plot_power_histogram_heatmap(
        group_h_daily,
        title=None,#f'Histogram over time, {fc} MHz, site {site}',
        bounds=HISTOGRAM_BOUNDS,
        ax=ax2,
        vmax=vmax
    )
    ax2.yaxis.get_label().set_visible(False)

    # set the metadata, but hide because it's redundant with the caption text
    fig.suptitle(f'Histograms {fc} MHz')
    fig._suptitle.set_visible(False)

    if fc== fc_max:
        reason = f'The peak band power at this frequency was the greatest among {group_name} at this site'
    else:
        reason = f'The peak band power at this frequency was the smallest among {group_name} at this site'

    set_caption(fig,
        f"""
        Histogram heat maps of band power at {fc} MHz from site {site} across the full data time span,
        binned by time of day ({TOD_BIN_SIZE_LABEL} bin, left) and and date ({DATE_BIN_SIZE_LABEL} bin, right). {reason}.
        """
    )

figures.plot_counts_each_frequency(hists_normed, fc_list, (-20,20));
xlabel(f'Power relative to 15-min histogram peak (dB')
title(f'{group_name} relative distribution', visible=False)
set_caption(
    gcf(),
    f"""Band power samples in {group_name} relative to the site noise and interference level at site {site},
    as gauged by the peak histogram level in each dwell window.
    """
)

h_rcum = histogram_analysis.rcumsum(hists.loc[[fc_map[fc] for fc in fc_list]]).sort_index()

fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth)

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=True,
    ax=ax1
);

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=False,
    ax=ax2,
    title=f'Site {site}'
);

ax2.yaxis.get_label().set_visible(False)
ax1.get_legend().set_visible(False)

fig.suptitle(f'Empirical SF {group_name}', visible=False)

set_caption(
    fig,
    f"""Empirical survival functions of band power in {group_name} at site {site}, shown first on log scale
    to illustrate rare events (left)
    and then on a linear scale (right) to emphasize the most frequent power levels. Dotted lines
    indicate expected distortion of the power level by 1\,dB due to noise or compression inside the sensor.
    """
)

### 2.4 GHz ISM Band

In [ ]:
fc_list = fc_ism
group_name = 'ISM bands'

h_group = histogram_analysis.loc_by_fc_spec(hists, fc_list)

fc_maxima = np.abs(h_group-1).idxmin(axis=1).groupby('Frequency').max()
fc_min, fc_max = fc_maxima.idxmin().round(1), fc_maxima.idxmax().round(1)

for fc in (fc_min, fc_max):
    fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth, sharey=True)

    group_h_tod = hists_by_tod.loc[fc_map[fc]].sort_index()
    group_h_daily = daily_hists.loc[fc_map[fc]]

    vmax = max(group_h_tod.max().max(), group_h_daily.max().max())

    figures.plot_power_histogram_heatmap(
        group_h_tod,
        title=None,
        bounds=HISTOGRAM_BOUNDS,
        ax=ax1,
        vmax=vmax,
        cbar=False
    )
    ax2.yaxis.get_label().set_visible(False)

    figures.plot_power_histogram_heatmap(
        group_h_daily,
        title=None,#f'Histogram over time, {fc} MHz, site {site}',
        bounds=HISTOGRAM_BOUNDS,
        ax=ax2,
        vmax=vmax
    )
    ax2.yaxis.get_label().set_visible(False)

    # set the metadata, but hide because it's redundant with the caption text
    fig.suptitle(f'Histograms {fc} MHz')
    fig._suptitle.set_visible(False)

    if fc== fc_max:
        reason = f'The peak band power at this frequency was the greatest among {group_name} at this site'
    else:
        reason = f'The peak band power at this frequency was the smallest among {group_name} at this site'

    set_caption(fig,
        f"""
        Histogram heat maps of band power at {fc} MHz from site {site} across the full data time span,
        binned by time of day ({TOD_BIN_SIZE_LABEL} bin, left) and and date ({DATE_BIN_SIZE_LABEL} bin, right). {reason}.
        """
    )

# figures.plot_counts_each_frequency(hists_normed, fc_list, (-20,20));
# xlabel(f'Power relative to 15-min histogram peak (dB')
# title(f'{group_name} relative distribution', visible=False)
# set_caption(
#     gcf(),
#     f"""Band power samples in {group_name} relative to the site noise and interference level at site {site},
#     as gauged by the peak histogram level in each dwell window.
#     """
# )

h_rcum = histogram_analysis.rcumsum(h_group).sort_index()

fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth)

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=True,
    ax=ax1
);

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=False,
    ax=ax2,
    title=f'Site {site}'
);

ax2.yaxis.get_label().set_visible(False)
ax1.get_legend().set_visible(False)

fig.suptitle(f'Empirical SF {group_name}', visible=False)

set_caption(
    fig,
    f"""Empirical survival functions of band power in {group_name} at site {site}, shown first on log scale
    to illustrate rare events (left)
    and then on a linear scale (right) to emphasize the most frequent power levels. Dotted lines
    indicate expected distortion of the power level by 1\,dB due to noise or compression inside the sensor.
    """
)

### 2695 MHZ Quiet Band

In [ ]:
for fc in [fc_quiet]:
    fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth, sharey=True)

    group_h_tod = hists_by_tod.loc[fc_map[fc]].sort_index()
    group_h_daily = daily_hists.loc[fc_map[fc]]

    vmax = max(group_h_tod.max().max(), group_h_daily.max().max())

    figures.plot_power_histogram_heatmap(
        group_h_tod,
        title=None,
        bounds=HISTOGRAM_BOUNDS,
        ax=ax1,
        vmax=vmax,
        cbar=False
    )
    ax2.yaxis.get_label().set_visible(False)

    figures.plot_power_histogram_heatmap(
        group_h_daily,
        title=None,#f'Histogram over time, {fc} MHz, site {site}',
        bounds=HISTOGRAM_BOUNDS,
        ax=ax2,
        vmax=vmax
    )
    ax2.yaxis.get_label().set_visible(False)

    # set the metadata, but hide because it's redundant with the caption text
    fig.suptitle(f'Histograms {fc} MHz')
    fig._suptitle.set_visible(False)

    # if fc== fc_max:
    #     reason = f'The peak band power at this frequency was the greatest among {group_name} at this site'
    # else:
    #     reason = f'The peak band power at this frequency was the smallest among {group_name} at this site'

    set_caption(fig,
        f"""
        Histogram heat maps of band power at {fc} MHz from site {site} across the full data time span,
        binned by time of day ({TOD_BIN_SIZE_LABEL} bin, left) and and date ({DATE_BIN_SIZE_LABEL} bin, right). {reason}.
        """
    )

ax = figures.plot_counts_each_frequency(hists, [fc_quiet], (-140,-60));

ax.set_title(f'Histogram 2695 MHz', visible=False)
# ax.get_title().set_visible(False)
set_caption(
    ax.get_figure(),
    f"""Histogram of band power measured in the 2695 MHz passive service band at site {site}.
    """
)

### 5 GHz U-NII1 Bands

In [ ]:
fc_list = fc_unii
group_name = 'U-NII bands'

h_group = histogram_analysis.loc_by_fc_spec(hists, fc_list)

fc_maxima = np.abs(h_group-1).idxmin(axis=1).groupby('Frequency').max()
fc_min, fc_max = fc_maxima.idxmin().round(1), fc_maxima.idxmax().round(1)

for fc in (fc_min, fc_max):
    fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth, sharey=True)

    group_h_tod = hists_by_tod.loc[fc_map[fc]].sort_index()
    group_h_daily = daily_hists.loc[fc_map[fc]]

    vmax = max(group_h_tod.max().max(), group_h_daily.max().max())

    figures.plot_power_histogram_heatmap(
        group_h_tod,
        title=None,
        bounds=HISTOGRAM_BOUNDS,
        ax=ax1,
        vmax=vmax,
        cbar=False
    )
    ax2.yaxis.get_label().set_visible(False)

    figures.plot_power_histogram_heatmap(
        group_h_daily,
        title=None,#f'Histogram over time, {fc} MHz, site {site}',
        bounds=HISTOGRAM_BOUNDS,
        ax=ax2,
        vmax=vmax
    )
    ax2.yaxis.get_label().set_visible(False)

    # set the metadata, but hide because it's redundant with the caption text
    fig.suptitle(f'Histograms {fc} MHz')
    fig._suptitle.set_visible(False)

    if fc== fc_max:
        reason = f'The peak band power at this frequency was the greatest among {group_name} at this site'
    else:
        reason = f'The peak band power at this frequency was the smallest among {group_name} at this site'

    set_caption(fig,
        f"""
        Histogram heat maps of band power at {fc} MHz from site {site} across the full data time span,
        binned by time of day ({TOD_BIN_SIZE_LABEL} bin, left) and and date ({DATE_BIN_SIZE_LABEL} bin, right). {reason}.
        """
    )

# figures.plot_counts_each_frequency(hists_normed, fc_list, (-20,20));
# xlabel(f'Power relative to 15-min histogram peak (dB')
# title(f'{group_name} relative distribution', visible=False)
# set_caption(
#     gcf(),
#     f"""Band power samples in {group_name} relative to the site noise and interference level at site {site},
#     as gauged by the peak histogram level in each dwell window.
#     """
# )

h_rcum = histogram_analysis.rcumsum(hists.loc[[fc_map[fc] for fc in fc_list]]).sort_index()

fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth)

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=True,
    ax=ax1
);

figures.plot_survival_each_frequency(
    h_rcum,
    noise_survival=shielded_noise_survival,
    noise_margin_dB=survival_noise_margin_dB,
    fc_list=fc_list,
    xlim=SF_BOUNDS[group_name],
    norm='all',
    logy=False,
    ax=ax2,
    title=f'Site {site}'
);

ax2.yaxis.get_label().set_visible(False)
ax1.get_legend().set_visible(False)

fig.suptitle(f'Empirical SF {group_name}', visible=False)

set_caption(
    fig,
    f"""Empirical survival functions of band power in {group_name} at site {site}, shown first on log scale
    to illustrate rare events (left)
    and then on a linear scale (right) to emphasize the most frequent power levels. Dotted lines
    indicate expected distortion of the power level by 1\,dB due to noise or compression inside the sensor.
    """
)

## Sample distributions by frequency group

In [ ]:
# remap frequency histograms into group histograms
histogram_analysis = reload(histogram_analysis)

group_mapping = dict(
    [(fc_map[fc], 'LTE uplink') for fc in fc_lte_ul]+
    [(fc_map[fc], 'LTE downlink') for fc in fc_lte_dl]+
    [(fc_map[fc], 'ISM') for fc in fc_ism]+
    [(fc_map[fc], 'Quiet') for fc in [fc_quiet]]+
    [(fc_map[fc], 'U-NII') for fc in fc_unii]
)

hist_agg = hists.groupby([group_mapping], level=['Frequency'], sort=False).sum()

# hist_normed_agg = aggregate_hist_by_frequency_group(hists_normed_total)
# hist_normed_agg.plot(logy=True, rasterized=True)
# xlabel(f'Power above median (dB)')
# ylabel(f'Fraction of power aperture samples in bands');
# xlim([-60,+70])

# hist_agg = aggregate_hist_by_frequency_group(hists.groupby(['Frequency','Site'], sort=False).sum());

fig, (ax1, ax2) = subplots(1,2,figsize=figsize_fullwidth)

(
    hist_agg
    .groupby('Frequency')
    .sum()
    .T
    .divide(1e6)
    .plot(logy=False, rasterized=True,ax=ax1,legend=False)
)
ax1.set_xlabel(figures.power_label);
ax1.set_ylabel(f'Sample count (millions)');
ax1.set_xlim(HISTOGRAM_BOUNDS);

# h_rcum_agg = hist_agg.iloc[::-1].cumsum().iloc[::-1].astype('float')
# h_rcum_agg.values[:] = h_rcum_agg.values[:]/h_rcum_agg.values[0,:][np.newaxis,:]

(
    hist_agg
    .groupby('Frequency').sum()
    .pipe(histogram_analysis.rcumsum, power_fix=False)
    .T
    .pipe(histogram_analysis.normalize_by_first_row)
    .plot(logy=True, rasterized=True,ax=ax2,legend=False)
)
# # h_rcum_agg.values[:] = h_rcum_agg.values[:]/h_rcum_agg.values[0,:][np.newaxis,:]
# h_rcum_agg.plot(logy=True, rasterized=True);
ax2.set_xlabel(figures.power_label);
ax2.set_ylabel(f'Fraction of samples > band power');
ax2.set_xlim(list(SF_BOUNDS.values())[0]);
figlegend(hist_agg.index.unique())

fig.suptitle(f'Empirical distributions by band type', visible=False)
set_caption(
    fig,
    f"""Histograms (left) and empirical survival functions (right) of all samples received
    in site {site} in each type of band under test.
    """
)